In [1]:
from src.v5.problem import ConstraintSystem, symbolic

### Demo 1

In [2]:
x, y, z, a, b = symbolic("x", "y", "z", "a", "b")

system = ConstraintSystem()
system.add(
    x + y == a,
    b == x*y,
    x <= 5,
    y >= 2)

In [3]:
x0struct = system.stategen({'x':1, 'y': -1})

In [4]:
AS = system.interpretation(elimination_order=[a.expr,b.expr])

In [5]:
AS.analysis(x0struct)

tensor([-1.,  0.,  1., -1.], dtype=torch.float64)

In [6]:
system2 = ConstraintSystem()
system2.add(
    system,
    a == b**2 + x,
    z == x - y
    )

In [11]:
system2.interpretation(elimination_order=[a, 
                                   system.interpretation(elimination_order=[a, b], default_solver='scipy.fsolve')], parallel_analysis=[z], residuals=[a]) #eliminate_order = [a,b]

AttributeError: 'SymbolicExpression' object has no attribute 'analysis'

In [ ]:
#optimal_order(system)
system.formulate(eliminate_order = None, parallel_analysis=system.statevars) #eliminate_order = [a,b]

### Aero Mini

In [1]:
from modeling.gen6.api import ConstraintSystem, symbolic
from engine.torchengine import ipoptsolver
from engine.torchdata import print_formatted_table
import numpy as np
np.set_printoptions(formatter={'float': lambda x: "{:0.2f}".format(x).rstrip('0').rstrip('.')})

In [2]:
A,V,S,W,Ww,CL,Re,Cf,CD,D = symbolic('A','V','S','W','Ww','C_L','Re','C_f','C_D','D')
rho,mu,k,t,e,CDA,SwetS,W0,pi = 1.23, 1.78e-5, 1.2, 0.12, 0.96, 0.0306, 2.05, 4940, 3.14

Aero = ConstraintSystem([
    CL == 2*W/(rho*V**2*S),
    Re == rho*V/mu*(S/A)**(1/2),
    Cf == 0.074/Re**0.2,
    CD == CDA/S+k*Cf*SwetS+CL**2/(3.14*A*e),
    D == 0.5*rho*V**2*CD*S
])
Struct = ConstraintSystem([
    Ww == 45.42*S+8.71e-5*2.5/t*A**(3/2)*S**(1/2)*(W0*W)**(1/2),
    W == W0+Ww
])
AeroStruct = ConstraintSystem([
    Aero,
    Struct
]) #.minimize(D)

In [ ]:
faero = Aero.interpretation(elimination_order=Aero.statevars)

In [22]:
fstruct1 = Struct.interpretation(parallel_analysis=Struct.statevars)
fstruct2 = Struct.interpretation(elimination_order=Struct.statevars)

In [30]:
x0struct = Struct.stategen({'A': 10, 'V': 30, 'S': 1, 'W':1000, 'Ww': 100})

In [31]:
fstruct2(x0struct).numpy()

array([1, 172.96, 5112.96, 10])

In [32]:
xsol = fstruct1(x0struct)

In [36]:
print_formatted_table([xsol.numpy()], *Struct.indices)

S Ww      W        A 
1 338.437 5278.437 10


In [33]:
fstruct2(x0struct).numpy(), fstruct2(xsol).numpy(), xsol.numpy()

(array([1, 338.44, 5278.44, 10]),
 array([1, 338.44, 5278.44, 10]),
 array([1, 338.44, 5278.44, 10]))

In [34]:
x0 = Aero.stategen({'A': 10, 'V': 30, 'S': 1, 'W':1000})

In [35]:
indices, idxrev = Aero.indices
print_formatted_table([faero(x0).numpy()], indices, idxrev)

S C_D   C_f      D     A  C_L   Re     V  W   
1 0.151 5.08e-03 83.79 10 1.807 6.56e5 30 1000


## MDO architectures

### MDF

In [ ]:
AeroStruct.formulate(elimination_order=[
    Aero.interpretation(elimination_order=Aero.statevars), 
    Struct.interpretation(parallel_analysis=Struct.statevars)])

### IDF

In [ ]:
AeroStruct.formulate(parallel_analysis=[Aero, Struct], interpretations=[
    Aero.interpretation(elimination_order=Aero.statevars), 
    Struct.interpretation(parallel_analysis=Struct.statevars)])

### AAO

In [ ]:
AeroStruct.formulate(residuals=AeroStruct.statevars)

### Sellar

In [ ]:
x, y1, y2, z1, z2 = symbolic("x", "y1", "y2", "z1", "z2")
D1 = ConstraintSystem([y1 == z1**2+z2+x-0.2*y2])
D2 = ConstraintSystem([y2 == y1**0.5+z1+z2])
C = ConstraintSystem([
    D1,  # C subset D1
    D2,  # C subset D2
    3.16 <= y1,
    y2 >= 24.0,
    0 <= x <= 10,
    0 <= z1 <= 10,
    0 <= z2 <= 10
]).minimize(x**2+z2+y1+sp.exp(-y2))
# D1.formulate()
# D2.formulate()
# C.formulate(eliminate_order=[D1, D2])
P = C.formulate(parallel_analysis=[D1, D2]) # -> formulation graph?
P.solve(x0 = P.stategen())